In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
####################################################################################
#    Aggregates SNR (.dat), latency (.dat), and ROI photos (png)
#            scraped from PhotoZ
#    Given ROI files in format slice#_rois.dat 
# 8/23/23 We want to add:
#     Take into account the RELATIVE STIM LOCATION within every barrel. Does velocity depend on distance from stimulating location?
#            New column: distance (um) from stim loc to relevant barrel pair boundary (1:2)
#            New column: distance (um) from stim loc to relevant barrel pair boundary (2:3)
#
#     METHOD
#         To calculate stim location to boundary, take these files produced by MovieMaker2.ipynb:
#             electrode.dat
#             corners_layer.dat
#             corners_barrel.dat
#         Calculate: Distance to lower barrel, distance to higher barrel,
#                    Distance to lower layer, distance to higher layer
#                       (if applicable for each)
####################################################################################

In [2]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
cell_type_prefix = 'scnn1a'
csv_prefix = cell_type_prefix + '_all_slices.csv'

enable_photoZ_interaction = True
initialize_photoZ = False
snr_cutoff_signal = 5.0
pixel_to_um_conversion = 480 / 80 # um per pixel

# Load reference data for all slices
reference_file = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/" + cell_type_prefix +"_catalog.csv"
reference_df = pd.read_csv(reference_file)
ref_cols = ['Age', 
          'Sex', 
          "Infusion", 
          'Stim (uA)']

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_prefix)
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Slice Location,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,...,abs(delta 1),abs(delta 2),Distance to Barrel 1,Distance to Barrel 2,Distance to Layer n1,Distance to Layer n2,Distance to Barrel A,Distance to Barrel B,Distance to Layer nA,Distance to Layer nB
0,2/8/2023,02_01_02,NaN,L4,47.7369,49.4509,49.8291,1.551910,4.440250,3.035860,...,NaN,NaN,NaN,NaN,NaN,NaN,215.228351,174.000000,180.093736,191.535134
1,2/10/2023,04_01_07,NaN,L4,52.7236,49.9505,51.1170,6.207490,13.131100,2.274550,...,NaN,NaN,NaN,NaN,NaN,NaN,151.086327,213.337695,149.891961,202.354632
2,2/14/2023,02_01_01,NaN,L4,47.5447,49.7343,49.9818,2.209280,2.558190,5.163110,...,NaN,NaN,NaN,NaN,NaN,NaN,198.974687,232.730513,132.022414,229.523778
3,2/14/2023,03_01_03,NaN,L4,51.1559,50.3250,51.2512,4.770840,12.652100,6.629770,...,NaN,NaN,NaN,NaN,NaN,NaN,228.000000,162.000000,159.011987,199.155462
4,2/28/2023,04_01_06,NaN,L2/3,55.0473,52.3564,51.1973,1.274260,3.233650,4.785560,...,NaN,NaN,NaN,NaN,NaN,NaN,189.026639,198.000000,206.933090,142.367160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,6/28/2023,02_04_03,Posterior,L4,96.0000,57.9622,54.0523,0.000000,0.415489,0.306664,...,NaN,NaN,NaN,NaN,NaN,NaN,45.334980,103.683397,54.966572,139.137831
74,6/28/2023,03_01_10,Posterior,L4,50.9643,96.0000,96.0000,1.093020,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,89.704987,123.158200,257.644879,39.925056
75,6/28/2023,03_02_03,Posterior,L4,59.7131,96.0000,50.4657,0.080140,0.000000,0.730507,...,NaN,NaN,NaN,NaN,NaN,NaN,134.482759,102.498405,93.155627,118.731332
76,6/28/2023,03_03_01,Posterior,L4,96.0000,50.7756,57.9677,0.000000,1.992940,0.410993,...,NaN,NaN,NaN,NaN,NaN,NaN,67.012847,143.255826,23.814860,207.587278


In [3]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()
    pa.alert("Please maximize PhotoZ before continuing.")

In [5]:
for subdir, dirs, files in os.walk(data_dir):
    zda_files = []
    roi_files = {}  # map slice number to ROIs file
    
    if 'hidden' in subdir:
        continue
    
    if 'selected_zda' not in subdir:
        continue
    
    # locate important file names
    for f in files:
        if f.endswith(".zda"):
            zda_files.append(f)
        if 'layer4_barrels_slice' in f[-35:] and f.endswith('_rois.dat'):
            slice_no, loc_no = None, None
            try:
                x = f.split("_rois.dat")[0]
                slice_no = int(x[-3])
                loc_no = int(x[-1])
            except Exception as e:
                print(e)
                print("Invalid ROI file:", f)
                slice_no = int(x[-1])
                loc_no = 0  # location 0 is fall back for ALL locations for this slice
            if slice_no not in roi_files:
                roi_files[slice_no] = {}
            roi_files[slice_no][loc_no] = f
    
    if len(roi_files) < 1 or len(zda_files) < 1:
        continue
    
    for zda_file in zda_files:
        rec_id = zda_file.split('.')[0]
        slice_no, loc_no, rec_no = [int(x) for x in rec_id.split("_")]
            
        # determine date and slice, loc, rec
        
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            aPhz.open_zda_file(subdir + "/" + zda_file)
            
        # reach into analysis##_##_## subdir (created by MovieMaker2)
        analysis_subdir = subdir + '/analysis' + rec_id
        corners_barrel_file = analysis_subdir + "/corners_barrel.dat"
        corners_layer_file = analysis_subdir + "/corners_layer.dat"
        electrode_file = analysis_subdir + "/electrode.dat"
        
        # read in barrel and layer axes data
        barrel_layer_axes = []
        for la_file in [corners_barrel_file, corners_layer_file]:
            print("\t", la_file)
            with open(la_file, 'r') as f:
                lines = f.readlines() 
            corners = [int(x) for x in lines[4:]] # the last 4 lines are diode numbers of corners
            print("\t", corners)
            layer_axes = LayerAxes(corners, verbose=False)
            print("\t", layer_axes)
            laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
            barrel_layer_axes.append(laminar_axis)
            barrel_layer_axes.append(laminar_axis_2)
        
        # open electrode as a single integer (its diode number) in variable stim_pt
        with open(electrode_file, 'r') as f:
            lines = f.readlines()
        stim_pt = int(lines[-1]) # last line is always electrode location
        aux_obj = LaminarROI([stim_pt]).get_points()
        stim_pt = aux_obj[0]  # should be a list of len 2, representing px location [x, y]
        
        # compute perpendicular distances between each of the 4 lines and the electrode point
        perp_dists = [line_obj.get_distance_to_point(stim_pt) * pixel_to_um_conversion for line_obj in barrel_layer_axes]
        
        date = subdir.split("selected_zda")[0][-9:-1]
        date = [int(x) for x in date.split("-")]
        date[2] += 2000  # full year format
        date = "/".join([str(d) for d in date])
        
        slice_no = int(rec_id.split("_")[0])
        roi_file = None
        if loc_no in roi_files[slice_no]:
            roi_file = roi_files[slice_no][loc_no]
        else:
            roi_file = roi_files[slice_no][0]
        print("\n\nscraping data from ", zda_file, "using", roi_file, "on date:", date)

        filename_end = "_" + rec_id + '.dat'

        # open roi file in photoZ
        if enable_photoZ_interaction:
            aPhz.select_roi_tab()
            aPhz.open_roi_file(subdir + "/" + roi_file)
            time.sleep(5)

        # save latency values from PhotoZ
        latency_filename = subdir + "/" + "latency_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_latency_trace_value()
            aPhz.save_trace_values(latency_filename)

        # save SNR values from PhotoZ
        snr_filename = subdir + "/" + "SNR_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_SNR_trace_value()
            aPhz.save_trace_values(snr_filename)

        # take JPG photo of ROIs so we know which measurements go where.
        rois_frame_filename = subdir + "/" + "ROIs_" + rec_id + ".jpg"
        if enable_photoZ_interaction:
            aPhz.save_map_jpeg(rois_frame_filename)
            
        
        # hit 426 x 77 to clear ROIs then select analog input channel 4 for stim latency
        x_ai, y_ai = 426, 77
        aPhz.click_location(x_ai, y_ai, button='right')
        aPhz.click_location(x_ai, y_ai, button='left')
        stim_time_filename = subdir + "/" + "stim_time_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_latency_trace_value()
            aPhz.save_trace_values(stim_time_filename)

        intermed_files = [latency_filename, snr_filename, rois_frame_filename]
        column_groups = [
            ['L4 barrel 1 Latency', 'L4 barrel 2 Latency', 'L4 barrel 3 Latency'],
            ['L4 barrel 1 SNR', 'L4 barrel 2 SNR', 'L4 barrel 3 SNR'],
            ['ROI jpg']
        ]
        
        # look up stim location, age, sex, infusion from cell_type_prefix_all.csv
        ref_row = reference_df[reference_df["Date"] == date][ref_cols]        
        new_row = {"Date" : date, 
                   "Slice/Loc/Rec": rec_id}
        for col in ref_cols:
            new_row[col] = ref_row.iloc[0][col]
        # write perpendicular distances
        new_row['Distance to Barrel A'] = perp_dists[0]
        new_row['Distance to Barrel B'] = perp_dists[1]
        new_row['Distance to Layer nA'] = perp_dists[2]
        new_row['Distance to Layer nB'] = perp_dists[3]
        
        # open these various files and store them to the pandas dataframe
        # each barrel/value pair is a column in the target CSV
        for j in range(len(intermed_files)):
            if intermed_files[j].endswith(".dat"):
                intermed_df = pd.read_csv(intermed_files[j],
                                          sep='\t',
                                          header=None,
                                          names=['Index',  'Values'])
                for i in range(len(column_groups[j])):
                    lat_val = intermed_df['Values'][i]
                    new_row[column_groups[j][i]] = lat_val
            else:  # .jpg, store link as csv value
                new_row[column_groups[j][0]] = intermed_files[j]
                
        # read stim time (same for all in this recording)
        stim_time_df = pd.read_csv(stim_time_filename, sep='\t', header=None, names=['Index',  'Values'])
        stim_time_val = stim_time_df['Values'][0]
        new_row['Actual Stim Time (ms)'] = stim_time_val
        df = df.append(new_row, ignore_index=True)


invalid literal for int() with base 10: 'c'
Invalid ROI file: layer4_barrels_slice2_rois.dat
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\selected_zda/analysis02_01_01/corners_barrel.dat
	 [1848, 1996, 3371, 3676, 4888, 5274]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\selected_zda/analysis02_01_01/corners_layer.dat
	 [744, 5625, 848, 5897]


scraping data from  02_01_01.zda using layer4_barrels_slice2_rois.dat on date: 2/8/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\selected_zda/analysis02_01_02/corners_barrel.dat
	 [1848, 1996, 3371, 3676, 4888, 5274]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\selected_zda/analysis02_01_02/corners_layer.dat
	 [744, 5625, 848, 5897]


scraping data from  02_01_02.zda using layer4_barrels_slice2_rois.dat on date: 2/8/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\selected_zda/analysis02_01_04/corners_barrel.dat
	 [1848, 1996, 3371, 3676,

	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-14-23\selected_zda/analysis03_01_05/corners_barrel.dat
	 [1530, 2235, 4250, 4795]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-14-23\selected_zda/analysis03_01_05/corners_layer.dat
	 [745, 5541, 1104, 5816]


scraping data from  03_01_05.zda using layer4_barrels_slice3_rois.dat on date: 2/14/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-14-23\selected_zda/analysis03_01_10/corners_barrel.dat
	 [1530, 2235, 4250, 4795]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-14-23\selected_zda/analysis03_01_10/corners_layer.dat
	 [745, 5541, 1104, 5816]


scraping data from  03_01_10.zda using layer4_barrels_slice3_rois.dat on date: 2/14/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-14-23\selected_zda/analysis03_01_12/corners_barrel.dat
	 [1530, 2235, 4250, 4795]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-14-23\selected_zda/analysis03_01_12/corners_layer.da

	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-07-23\selected_zda/analysis01_05_05/corners_barrel.dat
	 [4069, 888, 5573, 3525]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-07-23\selected_zda/analysis01_05_05/corners_layer.dat
	 [1010, 4165, 5396, 2078]


scraping data from  01_05_05.zda using layer4_barrels_slice1_rois.dat on date: 3/7/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-07-23\selected_zda/analysis02_01_02/corners_barrel.dat
	 [1351, 5818, 913, 5445]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-07-23\selected_zda/analysis02_01_02/corners_layer.dat
	 [812, 2398, 3848, 4795]


scraping data from  02_01_02.zda using layer4_barrels_slice2_rois.dat on date: 3/7/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-07-23\selected_zda/analysis02_01_04/corners_barrel.dat
	 [1351, 5818, 913, 5445]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-07-23\selected_zda/analysis02_01_04/corners_layer.dat
	 

	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-15-23\selected_zda/analysis03_01_05/corners_barrel.dat
	 [5297, 820, 5576, 1340]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-15-23\selected_zda/analysis03_01_05/corners_layer.dat
	 [2810, 3117, 4808, 5354]


scraping data from  03_01_05.zda using layer4_barrels_slice3_rois.dat on date: 3/15/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-15-23\selected_zda/analysis03_01_08/corners_barrel.dat
	 [5297, 820, 5576, 1340]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-15-23\selected_zda/analysis03_01_08/corners_layer.dat
	 [2810, 3117, 4808, 5354]


scraping data from  03_01_08.zda using layer4_barrels_slice3_rois.dat on date: 3/15/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-15-23\selected_zda/analysis03_02_02/corners_barrel.dat
	 [5219, 582, 5493, 852]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/03-15-23\selected_zda/analysis03_02_02/corners_layer.dat


	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/05-11-23\selected_zda/analysis03_06_01/corners_barrel.dat
	 [4809, 5344, 3531, 4628, 2172, 3194, 1053, 2153]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/05-11-23\selected_zda/analysis03_06_01/corners_layer.dat
	 [833, 4740, 1176, 5403]


scraping data from  03_06_01.zda using layer4_barrels_slice3_rois.dat on date: 5/11/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/05-11-23\selected_zda/analysis03_06_02/corners_barrel.dat
	 [4809, 5344, 3531, 4628, 2172, 3194, 1053, 2153]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/05-11-23\selected_zda/analysis03_06_02/corners_layer.dat
	 [833, 4740, 1176, 5403]


scraping data from  03_06_02.zda using layer4_barrels_slice3_rois.dat on date: 5/11/2023
invalid literal for int() with base 10: 'c'
Invalid ROI file: layer4_barrels_slice1_rois.dat
invalid literal for int() with base 10: 'c'
Invalid ROI file: layer4_barrels_slice2_rois.dat
	 C:/Users/jjudge3/D

...

	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-28-23\selected_zda/analysis03_02_03/corners_barrel.dat
	 [2498, 2377, 5064, 4780]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-28-23\selected_zda/analysis03_02_03/corners_layer.dat
	 [1294, 4346, 1163, 4456]


scraping data from  03_02_03.zda using layer4_barrels_slice3_2_rois.dat on date: 9/28/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-28-23\selected_zda/analysis03_03_01/corners_barrel.dat
	 [2737, 2138, 4746, 4383]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-28-23\selected_zda/analysis03_03_01/corners_layer.dat
	 [1375, 5073, 1244, 5100]


scraping data from  03_03_01.zda using layer4_barrels_slice3_3_rois.dat on date: 9/28/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-28-23\selected_zda/analysis03_04_04/corners_barrel.dat
	 [2571, 2454, 4659, 4059]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-28-23\selected_zda/analysis03_04_04/corners_la

	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-10-23\selected_zda/analysis01_01_01/corners_barrel.dat
	 [3448, 4715, 2179, 3591]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-10-23\selected_zda/analysis01_01_01/corners_layer.dat
	 [927, 4497, 1352, 5407]


scraping data from  01_01_01.zda using layer4_barrels_slice1_1_rois.dat on date: 10/10/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-10-23\selected_zda/analysis01_02_01/corners_barrel.dat
	 [4862, 3928, 3591, 2410]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-10-23\selected_zda/analysis01_02_01/corners_layer.dat
	 [661, 4808, 1011, 4669]


scraping data from  01_02_01.zda using layer4_barrels_slice1_2_rois.dat on date: 10/10/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-10-23\selected_zda/analysis01_03_02/corners_barrel.dat
	 [5110, 4088, 3511, 2731]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-10-23\selected_zda/analysis01_03_02/corners_la

	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-17-23\selected_zda/analysis02_07_02/corners_barrel.dat
	 [925, 2311, 1611, 5108]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-17-23\selected_zda/analysis02_07_02/corners_layer.dat
	 [841, 3208, 1676, 5541]


scraping data from  02_07_02.zda using layer4_barrels_slice2_7_rois.dat on date: 10/17/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-17-23\selected_zda/analysis02_08_01/corners_barrel.dat
	 [1289, 5102, 929, 2149]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-17-23\selected_zda/analysis02_08_01/corners_layer.dat
	 [838, 3048, 1115, 5539]


scraping data from  02_08_01.zda using layer4_barrels_slice2_8_rois.dat on date: 10/17/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-17-23\selected_zda/analysis02_09_01/corners_barrel.dat
	 [1772, 5262, 761, 2390]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-17-23\selected_zda/analysis02_09_01/corners_layer

	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis02_05_02/corners_barrel.dat
	 [5468, 1064, 5576, 1413]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis02_05_02/corners_layer.dat
	 [2808, 3030, 5208, 4791]


scraping data from  02_05_02.zda using layer4_barrels_slice2_5_rois.dat on date: 10/19/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis02_06_02/corners_barrel.dat
	 [4986, 582, 5494, 1332]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis02_06_02/corners_layer.dat
	 [2648, 2877, 4808, 4315]


scraping data from  02_06_02.zda using layer4_barrels_slice2_6_rois.dat on date: 10/19/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis02_07_01/corners_barrel.dat
	 [5388, 984, 5576, 1577]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis02_07_01/corners_la

	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis05_03_01/corners_barrel.dat
	 [2654, 1110, 5391, 3437]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis05_03_01/corners_layer.dat
	 [745, 5565, 1019, 3269]


scraping data from  05_03_01.zda using layer4_barrels_slice5_3_rois.dat on date: 10/19/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis05_04_01/corners_barrel.dat
	 [2097, 1437, 5153, 4232]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis05_04_01/corners_layer.dat
	 [905, 5653, 1020, 3432]


scraping data from  05_04_01.zda using layer4_barrels_slice5_4_rois.dat on date: 10/19/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis05_05_01/corners_barrel.dat
	 [2342, 1037, 5308, 4069]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/10-19-23\selected_zda/analysis05_05_01/corners_la

In [8]:
# write the pandas dataframe back to csv, fill in the Stim Layer metadata, and read it back in to df
csv_filename = cell_type_prefix + '_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)
input("Press Enter once you have filled in the Stim Layer metadata in " + csv_filename)

Press Enter once you have filled in the Stim Layer metadata in scnn1a_all_slices-edited.csv


''

In [9]:
# load the all-data CSV dir after you've filled in the Stim location metadata
df = pd.read_csv(data_dir + csv_filename)  # from the edited csv
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Slice Edge,Most Medial Barrel,Medial Dir,Slice Location,Stim Location,Actual Stim Time (ms),Stim Barrel,L4 barrel 1 Latency,...,abs(delta 1),abs(delta 2),Distance to Barrel 1,Distance to Barrel 2,Distance to Layer n1,Distance to Layer n2,Distance to Barrel A,Distance to Barrel B,Distance to Layer nA,Distance to Layer nB
0,2/8/2023,02_01_02,NaN,NaN,NaN,NaN,L4,48.4226,NaN,47.7369,...,NaN,NaN,NaN,NaN,NaN,NaN,215.228351,174.000000,180.093736,191.535134
1,2/10/2023,04_01_07,NaN,NaN,NaN,NaN,L4,48.4429,NaN,52.7236,...,2.7731,NaN,NaN,NaN,NaN,NaN,151.086327,213.337695,149.891961,202.354632
2,2/14/2023,02_01_01,NaN,NaN,NaN,NaN,L4,48.4645,NaN,47.5447,...,NaN,NaN,NaN,NaN,NaN,NaN,198.974687,232.730513,132.022414,229.523778
3,2/14/2023,03_01_03,NaN,NaN,NaN,NaN,L4,48.4195,NaN,51.1559,...,NaN,0.9262,NaN,NaN,NaN,NaN,228.000000,162.000000,159.011987,199.155462
4,2/28/2023,02_01_01,NaN,NaN,NaN,NaN,L5,48.4151,NaN,50.3009,...,NaN,1.4949,NaN,NaN,NaN,NaN,44.346286,328.107622,171.794234,154.795349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,10/19/2023,05_06_02,down/right,1.0,down/left,Anterior,L2/3,48.4228,1.0,52.5402,...,1.2641,NaN,NaN,NaN,NaN,NaN,24.997803,181.431965,314.744870,103.797327
170,10/19/2023,06_01_01,down,3.0,right,Posterior,L4,48.5290,2.0,50.6833,...,0.3503,NaN,NaN,NaN,NaN,NaN,118.898651,202.376373,288.667605,69.296479
171,10/19/2023,06_02_02,down,3.0,right,Posterior,L4,48.4103,3.0,50.3022,...,0.7656,1.0368,NaN,NaN,NaN,NaN,151.005549,215.305522,75.027164,306.897033
172,10/19/2023,06_03_01,down,3.0,right,Posterior,L2/3,48.5911,3.0,52.3050,...,0.5673,NaN,NaN,NaN,NaN,NaN,322.923386,22.555042,70.935246,350.093066


In [15]:
# compute statistics on barrel pairs:
#    1) whether delta latency should be considered based on SNR
#    2) Delta latency between L4 barrels (b1 - b2, b3 - b2)
df['Delta latency 1 (barrel 1 - barrel 2)'] = df['L4 barrel 1 Latency'] - df['L4 barrel 2 Latency']
df['Delta latency 2 (barrel 3 - barrel 2)'] = df['L4 barrel 3 Latency'] - df['L4 barrel 2 Latency']

# if either barrel 1 or barrel 2 has low SNR, don't compute the delta latency between these two
df.loc[(df['L4 barrel 1 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 1 (barrel 1 - barrel 2)'] = None

# Actual latencies (instead of deltas)
df["Actual latency barrel 1"] =  df['L4 barrel 1 Latency'] - df['Actual Stim Time (ms)']
df["Actual latency barrel 2"] =  df['L4 barrel 2 Latency'] - df['Actual Stim Time (ms)']
df["Actual latency barrel 3"] =  df['L4 barrel 3 Latency'] - df['Actual Stim Time (ms)']
# apply SNR cutoffs for actual latencies
df.loc[((df['L4 barrel 1 SNR'] < snr_cutoff_signal) | (df['L4 barrel 1 Latency'] <= 0)),
       'L4 barrel 1 Latency'] = None
df.loc[((df['L4 barrel 2 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 Latency'] <= 0)),
       'L4 barrel 2 Latency'] = None
df.loc[((df['L4 barrel 3 SNR'] < snr_cutoff_signal) | (df['L4 barrel 3 Latency'] <= 0)),
       'L4 barrel 3 Latency'] = None

# same for barrel 2 or barrel 3
df.loc[(df['L4 barrel 3 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 2 (barrel 3 - barrel 2)'] = None

for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    print("Barrel pairs in", stim_loc)
    # count number of barrel pairs with no signal crossing
    n_barrel_no_cross = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                               (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 1 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 3 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_no_cross, "no crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with signal crossing
    n_barrel_cross = len(df2[(df2['L4 barrel 1 SNR'] > snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_cross += len(df2[(df2['L4 barrel 2 SNR'] > snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_cross, "crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with no signal
    n_barrel_none = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] < snr_cutoff_signal)])
    n_barrel_none += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] < snr_cutoff_signal)])
    print("\t", n_barrel_none, "no signal out of", len(df2) * 2, "in layer", stim_loc)

Barrel pairs in L2/3
	 30 no crossing out of 120 in layer L2/3
	 26 crossing out of 120 in layer L2/3
	 64 no signal out of 120 in layer L2/3
Barrel pairs in L4
	 62 no crossing out of 196 in layer L4
	 25 crossing out of 196 in layer L4
	 108 no signal out of 196 in layer L4
Barrel pairs in L5
	 12 no crossing out of 28 in layer L5
	 6 crossing out of 28 in layer L5
	 10 no signal out of 28 in layer L5


In [16]:
df['abs(delta 1)'] = np.abs(df['Delta latency 1 (barrel 1 - barrel 2)'])
df['abs(delta 2)'] = np.abs(df['Delta latency 2 (barrel 3 - barrel 2)'])



for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    dfd1 = df2['abs(delta 1)'].dropna()
    dfd2 = df2['abs(delta 2)'].dropna()

    print("Latency between barrel pairs in", stim_loc)
    dfds = np.concatenate([dfd1, dfd2])
    print("\t", 
          str(np.mean(dfds))[:5], 
          "+/-", 
          str(np.std(dfds))[:5], 
          "(n =", 
          str(len(dfds)) + ")")
    print("\t", dfds)

Latency between barrel pairs in L2/3
	 0.690 +/- 0.619 (n = 26)
	 [1.3583 1.4885 0.2138 0.0093 0.1229 0.0447 1.1051 0.3143 0.1379 1.2641
 0.5673 2.4054 1.6877 0.7967 0.2299 0.7214 0.2692 1.2911 0.5468 0.0567
 0.1857 0.145  0.1196 1.252  1.0902 0.5346]
Latency between barrel pairs in L4
	 1.028 +/- 1.001 (n = 25)
	 [2.7731 0.0717 1.222  0.1611 1.3073 1.6698 0.4578 0.695  1.9031 0.3503
 0.7656 0.9262 0.957  1.3192 0.0676 2.1705 0.1656 0.7253 0.853  0.543
 0.3128 4.6423 0.1724 0.4323 1.0368]
Latency between barrel pairs in L5
	 0.764 +/- 0.546 (n = 6)
	 [0.1239 0.5704 1.4949 0.5802 1.5165 0.3024]


In [17]:
# write the pandas dataframe back to csv
csv_filename = cell_type_prefix + '_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [18]:
df

,Date,Slice/Loc/Rec,Slice Edge,Most Medial Barrel,Medial Dir,Slice Location,Stim Location,Actual Stim Time (ms),Stim Barrel,L4 barrel 1 Latency,...,Distance to Barrel 2,Distance to Layer n1,Distance to Layer n2,Distance to Barrel A,Distance to Barrel B,Distance to Layer nA,Distance to Layer nB,Actual latency barrel 1,Actual latency barrel 2,Actual latency barrel 3
0,2/8/2023,02_01_02,NaN,NaN,NaN,NaN,L4,48.4226,NaN,NaN,...,NaN,NaN,NaN,215.228351,174.000000,180.093736,191.535134,NaN,NaN,NaN
1,2/10/2023,04_01_07,NaN,NaN,NaN,NaN,L4,48.4429,NaN,52.7236,...,NaN,NaN,NaN,151.086327,213.337695,149.891961,202.354632,4.2807,1.5076,NaN
2,2/14/2023,02_01_01,NaN,NaN,NaN,NaN,L4,48.4645,NaN,NaN,...,NaN,NaN,NaN,198.974687,232.730513,132.022414,229.523778,NaN,NaN,1.5173
3,2/14/2023,03_01_03,NaN,NaN,NaN,NaN,L4,48.4195,NaN,NaN,...,NaN,NaN,NaN,228.000000,162.000000,159.011987,199.155462,NaN,1.9055,2.8317
4,2/28/2023,02_01_01,NaN,NaN,NaN,NaN,L5,48.4151,NaN,NaN,...,NaN,NaN,NaN,44.346286,328.107622,171.794234,154.795349,NaN,2.4255,0.9306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,10/19/2023,05_06_02,down/right,1.0,down/left,Anterior,L2/3,48.4228,1.0,52.5402,...,NaN,NaN,NaN,24.997803,181.431965,314.744870,103.797327,4.1174,2.8533,NaN
170,10/19/2023,06_01_01,down,3.0,right,Posterior,L4,48.5290,2.0,50.6833,...,NaN,NaN,NaN,118.898651,202.376373,288.667605,69.296479,2.1543,1.8040,NaN
171,10/19/2023,06_02_02,down,3.0,right,Posterior,L4,48.4103,3.0,50.3022,...,NaN,NaN,NaN,151.005549,215.305522,75.027164,306.897033,1.8919,2.6575,3.6943
172,10/19/2023,06_03_01,down,3.0,right,Posterior,L2/3,48.5911,3.0,52.3050,...,NaN,NaN,NaN,322.923386,22.555042,70.935246,350.093066,3.7139,3.1466,NaN
